In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, preprocessing
from tensorflow.keras.applications import MobileNetV2

# Download and preprocess the dataset
(train_data, validation_data), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)
num_classes = info.features['label'].num_classes

def preprocess_data(data):
    image = data['image']
    label = data['label']
    image = tf.image.resize(image, (224, 224))  # Resize the image
    image = tf.cast(image, tf.float32) / 255.0   # Normalize pixel values
    return image, tf.one_hot(label, num_classes) # One-hot encode labels

train_data = train_data.map(preprocess_data).shuffle(1000).batch(32)
validation_data = validation_data.map(preprocess_data).batch(32)

# Define a transfer learning model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=10, validation_data=validation_data)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_data)
print('\nTest accuracy:', test_acc)


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
115/115 [==============================] - 351s 3s/step - loss: 1.0189 - accuracy: 0.7326 - val_loss: 0.4582 - val_accuracy: 0.8506
Epoch 2/10
115/115 [==============================] - 348s 3s/step - loss: 0.2614 - accuracy: 0.9204 - val_loss: 0.4696 - val_accuracy: 0.8490
Epoch 3/10
115/115 [==============================] - 362s 3s/step - loss: 0.1489 - accuracy: 0.9571 - val_loss: 0.4261 - val_accuracy: 0.8605
Epoch 4/10
115/115 [==============================] - 283s 2s/step - loss: 0.0840 - accuracy: 0.9837 - val_loss: 0.4529 - val_accuracy: 0.8588
Epoch 5/10
115/115 [==============================] - 289s 2s/step - loss: 0.0597 - accuracy: 0.9886 - val_loss: 0.4520 - val_accuracy: 0.8662
Epoch 6/10
115/115 [==============================] - 283s 2s/step - loss: 0.0366 - accuracy: 0.9951 - val_loss: 0.4261 - val_accuracy: 0.8694
Epoch 7/10
115/115 [==============================] - 350s 3s/step - loss: 0.02

In [ ]:
model.save('animal_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
